In [1]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '...'

In [4]:
openai.api_key = os.environ["OPENAI_API_KEY"] 

Prolazimo kroz PDF fajl i sa svek stranice izvlacimo tekst. Varijabla tekst na kraju sadrzi sav tekst sa slike.

In [31]:
from PyPDF2 import PdfReader

file_path = 'C:/Users/enaf/Documents/OPENAI/Dokumenta za ChatGPT/CTSI BG Company Benefits_A4 26 07 2023.pdf'
text = ''
# Otvori PDF fajl
with open(file_path, 'rb') as pdf_file:
    pdf_reader =PdfReader(pdf_file)
    for page in pdf_reader.pages:
            text += page.extract_text()

To count the number of tokens that gpt-3.5-turbo will use for some text we need to initialize the tiktoken tokenizer

In [32]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [33]:
token_counts = tiktoken_len(text)
token_counts

1458

Nas dokument ima ukupno 1458 tokena.

In [34]:
text

'Flexible working hours \nThe schedule allows you to start your day between 8 and 10 AM.  \nOn Fridays, working hours are reduced to seven hours.  \nA hybrid workplace model \nOur colleagues from the Delivery and ICT team, who can complete some of their tasks from home, get \nto stay in for up to 8 days monthly.  \nDays off \nFathers have 5 days off when their child is born, and all parents get an additional day off for the \nfirst day of school. Furthermore, we have additional days off for different occasions like weddings, \nchildbirth, etc.  \nVacation \nOur employees can get up to 28 days of annual leave. The legal minimum of 20 days is increased \nbased on seniority, years of service, education and health and social status.  \n5 days of sick leave @ 100% salary \nAll employees have the right to call in sick, and even without the medical report, have up to 5 days of \nsuch sick leave annually, compensated at 100% pay.Flexible working modelCompany\nbenefits\nBelgrade\nThe big-ol’ te

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', '.']
)

In [54]:
chunks = text_splitter.split_text(text)
len(chunks)

11

In [55]:
chunks[0]

'Flexible working hours \nThe schedule allows you to start your day between 8 and 10 AM.  \nOn Fridays, working hours are reduced to seven hours.  \nA hybrid workplace model \nOur colleagues from the Delivery and ICT team, who can complete some of their tasks from home, get \nto stay in for up to 8 days monthly.  \nDays off \nFathers have 5 days off when their child is born, and all parents get an additional day off for the \nfirst day of school. Furthermore, we have additional days off for different occasions like weddings, \nchildbirth, etc.  \nVacation \nOur employees can get up to 28 days of annual leave. The legal minimum of 20 days is increased \nbased on seniority, years of service, education and health and social status.  \n5 days of sick leave @ 100% salary'

In [58]:
tiktoken_len(chunks[0])

176

In [56]:
chunks[1]

'based on seniority, years of service, education and health and social status.  \n5 days of sick leave @ 100% salary \nAll employees have the right to call in sick, and even without the medical report, have up to 5 days of \nsuch sick leave annually, compensated at 100% pay.Flexible working modelCompany\nbenefits\nBelgrade\nThe big-ol’ tech campus \nYes, we take pride in having a modern, green and spacious campus in the heart of New Belgrade.  \nWe share the campus with some of the other members of the Comtrade Group, and enjoy interacting \nwith our extended-family at the company park, new company café, or the restaurant.  \nParking space\nYou will always have a parking space in the Belgrade office, no matter how unrealistic it sounds.  \nNew & renovated workspace'

In [59]:
tiktoken_len(chunks[1])

173

In [57]:
chunks[2]

'with our extended-family at the company park, new company café, or the restaurant.  \nParking space\nYou will always have a parking space in the Belgrade office, no matter how unrealistic it sounds.  \nNew & renovated workspace \nOur colleagues get to keep their focus and feel good at all times, thanks to new or renovated modern \noffice space, contemporary equipment, comfortable ergonomic chairs and everything they need \nduring the working day. The Fun Room is an area in our Belgrade campus where employees can relax \nplaying table tennis, football, darts and chess.\nCompany restaurant \nGet a wide choice of at meals at the company restaurant at affordable prices, freshly prepared and \nadapted to different dietary regimes.\nFree snacks, fruits and drinks \nRunning low on energy? We got you covered. You will be able to have some snacks, fruits and drinks \navailable at the Fun Room.\nPrivate health insurance'

In [60]:
tiktoken_len(chunks[2])

182

Iako se nekada desi da se prekine chunk u pola recenice i da se izgubi znacenje, sa overlap 
parametrom omogucavamo da pocetak narednog chunka bude 50 tokena iza kraja i da se uhvati semantika isecene recenice.